# Clustering the twitter samples corpus

**corpushash** is a simple library that aims to make the natural language processing of sensitive documents easier. the library enables performing common NLP tasks on sensitive documents without disclosing their contents. This is done by hashing every token in the corpus along with a salt (to prevent dictionary attacks). 

its workflow is as simple as having the sensitive corpora as a python nested list (or generator) whose elements are themselves (nested) lists of strings. after the hashing is done, NLP can be carried out by a third party, and when the results are in they can be decoded by a dictionary that maps hashes to the original strings. so that makes:

```python
import corpushash as ch
hashed_corpus = ch.CorpusHash(mycorpus_as_a_nested_list, '/home/sensitive-corpus')
>>> "42 documents hashed and saved to '/home/sensitive-corpus/public/$(timestamp)'"
```
**NLP is done, and `results` are in**:
```python
for token in results:
    print(token, ">", hashed_corpus.decode_dictionary[token])
>>> "7)JBMGG?sGu+>%Js~dG=%c1Qn1HpAU{jM-~Buu7?" > "gutenberg"

```

#### loading libraries

In [10]:
import gensim
import logging, bz2, os
from corpushash import CorpusHash
from nltk.corpus import twitter_samples as tt
import numpy as np
import string
from gensim import corpora, models, similarities

In [11]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#### uncomment this if you don't have the corpus downloaded

In [12]:
#import nltk
#nltk.download('twitter_samples')

specify the directory you'd like to save files to:

In [13]:
path = os.getcwd()

this is needed because `gensim`'s `doc2bow` has some random behaviour:

In [14]:
np.random.seed(42)

### the twitter samples corpus

this is how the corpus looks like:

In [15]:
tt.strings()[:10]

['hopeless for tmr :(',
 "Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(",
 '@Hegelbon That heart sliding into the waste basket. :(',
 '“@ketchBurning: I hate Japanese call him "bani" :( :(”\n\nMe too',
 'Dang starting next week I have "work" :(',
 "oh god, my babies' faces :( https://t.co/9fcwGvaki0",
 '@RileyMcDonough make me smile :((',
 '@f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln',
 'why?:("@tahuodyy: sialan:( https://t.co/Hv1i0xcrL2"',
 'Athabasca glacier was there in #1948 :-( #athabasca #glacier #jasper #jaspernationalpark #alberta #explorealberta #… http://t.co/dZZdqmf7Cz']

but we'll be using the pre-tokenized version:

In [16]:
tt.tokenized()[0]

['hopeless', 'for', 'tmr', ':(']

In [17]:
len(tt.tokenized())

30000

In [18]:
decoded_twitter = tt.tokenized()

#### building gensim dictionary

from this document generator gensim will build a dictionary that maps every hashed token to an ID, a mapping which is later used to calculate the tf-idf weights:

In [19]:
id2word = gensim.corpora.Dictionary(decoded_twitter)

2017-05-23 22:39:18,669 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-05-23 22:39:20,438 : INFO : adding document #10000 to Dictionary(24343 unique tokens: ['Americano', 'http://t.co/lFm9Zq4Tj2', 'meditation', 'Danny', '@nabilaAF2013']...)
2017-05-23 22:39:22,948 : INFO : adding document #20000 to Dictionary(35614 unique tokens: ['Americano', 'Roche', 'http://t.co/lFm9Zq4Tj2', 'Republican', 'meditation']...)
2017-05-23 22:39:25,462 : INFO : built Dictionary(42532 unique tokens: ['https://t.co/aueMOZvKeq', 'Americano', '#Wow', 'Roche', 'http://t.co/lFm9Zq4Tj2']...) from 30000 documents (total 580322 corpus positions)


In [20]:
id2word[0]

'hopeless'

#### bag-of-words

to build a tf-idf model, the gensim library needs an input that yields this vectorized bag-of-words when iterated over:

In [21]:
mm = [id2word.doc2bow(text) for text in decoded_twitter]
gensim.corpora.MmCorpus.serialize(os.path.join(path, 'twitter_pt_tfidf.mm'), mm)

2017-05-23 22:39:30,548 : INFO : storing corpus in Matrix Market format to /home/bruno/Documents/github/corpushash/notebooks/twitter_pt_tfidf.mm
2017-05-23 22:39:30,552 : INFO : saving sparse matrix to /home/bruno/Documents/github/corpushash/notebooks/twitter_pt_tfidf.mm
2017-05-23 22:39:30,556 : INFO : PROGRESS: saving document #0
2017-05-23 22:39:30,753 : INFO : PROGRESS: saving document #1000
2017-05-23 22:39:30,962 : INFO : PROGRESS: saving document #2000
2017-05-23 22:39:31,170 : INFO : PROGRESS: saving document #3000
2017-05-23 22:39:31,367 : INFO : PROGRESS: saving document #4000
2017-05-23 22:39:31,557 : INFO : PROGRESS: saving document #5000
2017-05-23 22:39:31,753 : INFO : PROGRESS: saving document #6000
2017-05-23 22:39:31,957 : INFO : PROGRESS: saving document #7000
2017-05-23 22:39:32,157 : INFO : PROGRESS: saving document #8000
2017-05-23 22:39:32,369 : INFO : PROGRESS: saving document #9000
2017-05-23 22:39:32,568 : INFO : PROGRESS: saving document #10000
2017-05-23 22:3

In [22]:
%%time
if os.path.exists(os.path.join(path, 'twitter_tfidf_model')):
    tfidf = models.TfidfModel.load(os.path.join(path, 'twitter_tfidf_model'))
else:
    tfidf = models.TfidfModel(mm)
    tfidf.save('twitter_tfidf_model')

2017-05-23 22:39:38,788 : INFO : collecting document frequencies
2017-05-23 22:39:38,793 : INFO : PROGRESS: processing document #0
2017-05-23 22:39:38,989 : INFO : PROGRESS: processing document #10000
2017-05-23 22:39:39,318 : INFO : PROGRESS: processing document #20000
2017-05-23 22:39:39,652 : INFO : calculating IDF weights for 30000 documents and 42531 features (538552 matrix non-zeros)
2017-05-23 22:39:39,890 : INFO : saving TfidfModel object under twitter_tfidf_model, separately None
2017-05-23 22:39:39,920 : INFO : saved twitter_tfidf_model


CPU times: user 1.1 s, sys: 28 ms, total: 1.13 s
Wall time: 1.14 s


## Calculating the LSI model

The next step is to train the LSI model with a tfidf transformed corpus. So we will need yet another generator to yield the transformed corpus.

In [23]:
def tfidf_corpus_stream(corpus):
    for doc in corpus:
        yield tfidf[doc]

In [24]:
tfidf_corpus_s = tfidf_corpus_stream(mm)

In [25]:
if os.path.exists(os.path.join(path, 'twitter_lsi_model')):
    lsi = gensim.models.LsiModel.load(os.path.join(path, 'twitter_lsi_model'))
else:
    lsi = gensim.models.lsimodel.LsiModel(corpus=tfidf_corpus_s, id2word=id2word, num_topics=100)
    lsi.save(os.path.join(path, 'twitter_lsi_model'))

2017-05-23 22:39:40,046 : INFO : using serial LSI version on this node
2017-05-23 22:39:40,050 : INFO : updating model with new documents
2017-05-23 22:39:44,837 : INFO : preparing a new chunk of documents
2017-05-23 22:39:45,831 : INFO : using 100 extra samples and 2 power iterations
2017-05-23 22:39:45,836 : INFO : 1st phase: constructing (42532, 200) action matrix
2017-05-23 22:39:47,061 : INFO : orthonormalizing (42532, 200) action matrix
2017-05-23 22:40:12,259 : INFO : 2nd phase: running dense svd on (200, 20000) matrix
2017-05-23 22:40:19,609 : INFO : computing the final decomposition
2017-05-23 22:40:19,628 : INFO : keeping 100 factors (discarding 22.970% of energy spectrum)
2017-05-23 22:40:23,244 : INFO : processed documents up to #20000
2017-05-23 22:40:23,254 : INFO : topic #0(20.207): 0.246*""" + 0.207*"SNP" + 0.186*"Tories" + 0.174*"Miliband" + 0.171*"in" + 0.162*"is" + 0.160*"to" + 0.154*"of" + 0.146*"Sco" + 0.146*"…"
2017-05-23 22:40:23,263 : INFO : topic #1(17.533): -0

In [26]:
for n in range(17):
    print("====================")
    print("Topic {}:".format(n))
    print("Coef.\t Token")
    print("--------------------")
    for tok,coef in lsi.show_topic(n):
        print("{:.3}\t{}".format(coef,tok))

Topic 0:
Coef.	 Token
--------------------
0.313	"
0.181	Tories
0.171	preoccupied
0.171	inequality
0.171	@Tommy_Colc
0.171	wrote
0.167	Miliband
0.167	claiming
0.166	w
0.164	man
Topic 1:
Coef.	 Token
--------------------
0.247	SNP
-0.21	"
0.178	Sco
0.177	to
0.176	protect
0.176	lots
0.175	definitely
0.172	@NicolaSturgeon
0.172	rather
0.17	let
Topic 2:
Coef.	 Token
--------------------
0.194	the
0.175	.
-0.169	protect
-0.169	lots
-0.168	definitely
-0.168	Sco
0.159	a
0.159	%
0.148	I
-0.147	MPs
Topic 3:
Coef.	 Token
--------------------
-0.646	%
-0.319	-
-0.266	(
-0.22	)
-0.195	1
-0.131	CON
-0.13	LAB
-0.127	poll
-0.126	8
-0.125	34
Topic 4:
Coef.	 Token
--------------------
-0.236	thus
-0.236	ahem
-0.236	@thomasmessenger
-0.236	http://t.co/DkLwCwzhDA
-0.235	financial
-0.234	caused
-0.233	global
-0.233	crisis
-0.225	For
-0.225	overspent
Topic 5:
Coef.	 Token
--------------------
0.341	FT
0.321	(
-0.232	%
0.2	)
0.19	:(
0.177	Jonathan
0.177	Ford
0.177	writer
0.176	Boris
-0.162	'
Topic 6:
Coef.	

# LSI on the hashed corpus

Now all of the original documents have been hashed, and we can run the same analysis we ran with the plain corpus. 

In [27]:
np.random.seed(42)

## processing using the `corpushash` library

#### instatiating CorpusHash class, which hashes the provided corpus to the corpus_path:

In [28]:
%%time
hashed = CorpusHash(decoded_twitter, 'twitter')

2017-05-23 22:41:43,678 - corpushash.hashers - INFO - 30000 documents hashed and saved to twitter/public/2017-05-23_22-41-14-659510.
2017-05-23 22:41:43,678 : INFO : 30000 documents hashed and saved to twitter/public/2017-05-23_22-41-14-659510.


CPU times: user 24.9 s, sys: 3.85 s, total: 28.7 s
Wall time: 29 s


that is it. `corpushash`'s work is done.

#### building dictionary for the hashed corpus

In [29]:
id2word = gensim.corpora.Dictionary(hashed.read_hashed_corpus())

2017-05-23 22:41:43,707 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-05-23 22:41:48,381 : INFO : adding document #10000 to Dictionary(24343 unique tokens: ['yXWupV7(%Jh!0%xFvLM3KCn!Rr$+BsbDU$W$N&d_', '6KU!3F6?h!OF9NsnPE^;r9Rj8e)AqhO!PYKbM3I1', 'C2QO*QC}-Y2aYL9fLx%Vs?PB&9OrQD$tI2DTzc{b', '?l|xct~8Qo?l~`s8&#O_OzN6<GN!%MmH4a#J9U;M', '<K>XG@1gDWqVR831Pu(2>Drd^vJ@X6&v64cA&bGY']...)
2017-05-23 22:41:53,801 : INFO : adding document #20000 to Dictionary(35614 unique tokens: ['yXWupV7(%Jh!0%xFvLM3KCn!Rr$+BsbDU$W$N&d_', '?&<)40p-obP!%ppvV+NMcy5N&+J+4vkkDrw=`r)7', '6KU!3F6?h!OF9NsnPE^;r9Rj8e)AqhO!PYKbM3I1', 'C2QO*QC}-Y2aYL9fLx%Vs?PB&9OrQD$tI2DTzc{b', '?l|xct~8Qo?l~`s8&#O_OzN6<GN!%MmH4a#J9U;M']...)
2017-05-23 22:41:59,155 : INFO : built Dictionary(42532 unique tokens: ['yXWupV7(%Jh!0%xFvLM3KCn!Rr$+BsbDU$W$N&d_', 'HAUZdq60Hh-U@NAP|^&3i%rnpTXX1QD3zOt<I9?X', '?&<)40p-obP!%ppvV+NMcy5N&+J+4vkkDrw=`r)7', '6KU!3F6?h!OF9NsnPE^;r9Rj8e)AqhO!PYKbM3I1', 'C2QO*QC}-Y2aYL9fLx%Vs?PB&9OrQD$t

In [30]:
id2word[0]

'>)>{1_a)^MuyM>C0m&TpEi&OwdI=p=a>{46-ep!@'

In [31]:
mm = [id2word.doc2bow(text) for text in hashed.read_hashed_corpus()]
gensim.corpora.MmCorpus.serialize(os.path.join(path, 'hashed_twitter_pt_tfidf.mm'), mm)

2017-05-23 22:42:12,175 : INFO : storing corpus in Matrix Market format to /home/bruno/Documents/github/corpushash/notebooks/hashed_twitter_pt_tfidf.mm
2017-05-23 22:42:12,179 : INFO : saving sparse matrix to /home/bruno/Documents/github/corpushash/notebooks/hashed_twitter_pt_tfidf.mm
2017-05-23 22:42:12,183 : INFO : PROGRESS: saving document #0
2017-05-23 22:42:12,379 : INFO : PROGRESS: saving document #1000
2017-05-23 22:42:12,574 : INFO : PROGRESS: saving document #2000
2017-05-23 22:42:12,768 : INFO : PROGRESS: saving document #3000
2017-05-23 22:42:12,962 : INFO : PROGRESS: saving document #4000
2017-05-23 22:42:13,149 : INFO : PROGRESS: saving document #5000
2017-05-23 22:42:13,349 : INFO : PROGRESS: saving document #6000
2017-05-23 22:42:13,557 : INFO : PROGRESS: saving document #7000
2017-05-23 22:42:13,758 : INFO : PROGRESS: saving document #8000
2017-05-23 22:42:13,964 : INFO : PROGRESS: saving document #9000
2017-05-23 22:42:14,163 : INFO : PROGRESS: saving document #10000
2

In [32]:
%%time
if os.path.exists(os.path.join(path, 'hashed_twitter_tfidf_model')):
    tfidf = models.TfidfModel.load(os.path.join(path, 'hashed_twitter_tfidf_model'))
else:
    tfidf = models.TfidfModel(mm)
    tfidf.save(os.path.join(path, 'hashed_twitter_tfidf_model'))

2017-05-23 22:42:20,264 : INFO : collecting document frequencies
2017-05-23 22:42:20,268 : INFO : PROGRESS: processing document #0
2017-05-23 22:42:20,473 : INFO : PROGRESS: processing document #10000
2017-05-23 22:42:20,786 : INFO : PROGRESS: processing document #20000
2017-05-23 22:42:21,118 : INFO : calculating IDF weights for 30000 documents and 42531 features (538552 matrix non-zeros)
2017-05-23 22:42:21,348 : INFO : saving TfidfModel object under /home/bruno/Documents/github/corpushash/notebooks/hashed_twitter_tfidf_model, separately None
2017-05-23 22:42:21,377 : INFO : saved /home/bruno/Documents/github/corpushash/notebooks/hashed_twitter_tfidf_model


CPU times: user 1.12 s, sys: 8 ms, total: 1.12 s
Wall time: 1.12 s


The next step is to train the LSI model with a tfidf transformed corpus. So we will need yet another generator to yield the transformed corpus.

In [33]:
def tfidf_corpus_stream(corpus):
    for doc in corpus:
        yield tfidf[doc]

In [34]:
tfidf_corpus_s = tfidf_corpus_stream(mm)

## Calculating the LSI model

In [35]:
if os.path.exists(os.path.join(path, 'hashed_twitter_lsi_model')):
    lsih = gensim.models.LsiModel.load(os.path.join(path, 'hashed_twitter_lsi_model'))
else:
    lsih = gensim.models.lsimodel.LsiModel(corpus=tfidf_corpus_s, id2word=id2word, num_topics=100)
    lsih.save(os.path.join(path, 'hashed_twitter_lsi_model'))

2017-05-23 22:42:21,498 : INFO : using serial LSI version on this node
2017-05-23 22:42:21,503 : INFO : updating model with new documents
2017-05-23 22:42:25,736 : INFO : preparing a new chunk of documents
2017-05-23 22:42:26,726 : INFO : using 100 extra samples and 2 power iterations
2017-05-23 22:42:26,730 : INFO : 1st phase: constructing (42532, 200) action matrix
2017-05-23 22:42:27,956 : INFO : orthonormalizing (42532, 200) action matrix
2017-05-23 22:42:51,259 : INFO : 2nd phase: running dense svd on (200, 20000) matrix
2017-05-23 22:42:58,155 : INFO : computing the final decomposition
2017-05-23 22:42:58,160 : INFO : keeping 100 factors (discarding 22.970% of energy spectrum)
2017-05-23 22:43:01,784 : INFO : processed documents up to #20000
2017-05-23 22:43:01,796 : INFO : topic #0(20.207): 0.246*"+xy0wTTqhE|JZz*%3WH{1@1q9h<G<=uLOD^et9xA" + 0.207*"d>egDXR4GAFvN3>C#)E7D`$~a8Jrm^3Sk6!^KgLD" + 0.186*"kGAJg>G9k#tX-bv0@t`HG>lN^h{c3!)<j`xP7D7)" + 0.174*"UC&2ub*;{Z+u~QBI_5Nvm##j=vQ4Kb{

Let now look at the topics generated, decoding the hashed tokens using the `decode_dictionary`.

In [36]:
for n in range(17):
    print("====================")
    print("Topic {}:".format(n))
    print("Coef.\t Token")
    print("--------------------")
    for tok,coef in lsih.show_topic(n):
        tok = hashed.decode_dictionary[tok.strip()][0]
        print("{:.3}\t{}".format(coef,tok))

Topic 0:
Coef.	 Token
--------------------
0.313	"
0.181	Tories
0.171	preoccupied
0.171	inequality
0.171	@Tommy_Colc
0.171	wrote
0.167	Miliband
0.167	claiming
0.166	w
0.164	man
Topic 1:
Coef.	 Token
--------------------
0.247	SNP
-0.21	"
0.178	Sco
0.177	to
0.176	protect
0.176	lots
0.175	definitely
0.172	@NicolaSturgeon
0.172	rather
0.17	let
Topic 2:
Coef.	 Token
--------------------
0.194	the
0.175	.
-0.169	protect
-0.169	lots
-0.168	definitely
-0.168	Sco
0.159	a
0.159	%
0.148	I
-0.147	MPs
Topic 3:
Coef.	 Token
--------------------
-0.646	%
-0.319	-
-0.266	(
-0.22	)
-0.195	1
-0.131	CON
-0.13	LAB
-0.127	poll
-0.126	8
-0.125	34
Topic 4:
Coef.	 Token
--------------------
-0.236	thus
-0.236	ahem
-0.236	@thomasmessenger
-0.236	http://t.co/DkLwCwzhDA
-0.235	financial
-0.234	caused
-0.233	global
-0.233	crisis
-0.225	For
-0.225	overspent
Topic 5:
Coef.	 Token
--------------------
0.341	FT
0.321	(
-0.232	%
0.2	)
0.19	:(
0.177	Ford
0.177	Jonathan
0.177	writer
0.176	Boris
-0.162	'
Topic 6:
Coef.	

#### comparing the resulting topics we see that the NLP's results are the same regardless of which corpus we use, i.e., we can use hashed corpora to perform NLP tasks in a lossless manner.